# News Articles Categorisation


The steps to build the model are

- Data Exploration
- Text Processing (Cleaning)
- Feature Extraction
- Modelling
- Use the Model

In this notebook, we will be documenting our project codes for our industry sponso, Food Security Exchange (FSX). This will be written in python, with supplementary comments at each step to ensure that users can both comprehend and trust the results and output generated by the algorithms.

Our project mines trigger events related news articles from various sources, before classifying them according to predefined supply chain disruption categories using both a rule-based and ML-based approach. Following this, a predictive risk score is assigned to every article.

# Possible Reference


- https://www.analyticsvidhya.com/blog/2021/08/malawi-news-classification-an-nlp-project/

# Importing Libraries

In [1]:
import nltk

# nltk.download()

In [2]:
# Import libraries

import pandas as pd
import re 

import nltk 
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost 
from sklearn.metrics  import classification_report
from sklearn import metrics
import time



# Combining multiple excel files into one dataframe


In [13]:
import os
import pandas as pd
cwd = os.path.abspath('') 
files = os.listdir(cwd) 

In [14]:
cwd

'C:\\Users\\Darren\\Documents\\GitHub\\FSX\\News Classifier'

In [15]:
files

['(Reference) News_Articles_Categorization.ipynb',
 '.ipynb_checkpoints',
 'avalanche_straitstimes.csv',
 'contamination_straitstimes.csv',
 'CSV data',
 'dengue_straitstimes.csv',
 'earthquake_straitstimes.csv',
 'ebola_straitstimes.csv',
 'FYP Article Classification Codes.ipynb',
 'Influenza_straitstimes.csv',
 'limnic eruption_straitstimes.csv',
 'pandemic_straitstimes.csv',
 'SarS_straitstimes.csv',
 'sinkhole_straitstimes.csv',
 'tsunami_straitstimes.csv',
 'Unsafe_straitstimes.csv',
 'volcanic eruption_straitstimes.csv']

In [16]:
df = pd.DataFrame()
for file in files:
     if file.endswith('.csv'):
         df = df.append(pd.read_csv(file), ignore_index=True) 
df.head()

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name
0,0,Feb-22,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,geophysical event
1,1,Feb-22,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,geophysical event
2,2,Feb-22,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,geophysical event
3,3,Feb-22,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,geophysical event
4,4,Feb-21,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,geophysical event


In [17]:
df.shape

(3494, 9)

In [19]:
df['new_class_name'].value_counts()

idiosyncratic        1320
geophysical event    1174
pandemic             1000
Name: new_class_name, dtype: int64

# Data Exploration

In [20]:
category = list(df['new_class_name'].unique())
category

['geophysical event', 'idiosyncratic', 'pandemic']

# Text Preprocessing

In [21]:
def preprocess(text):
    
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the article
    Return:
      lem(list of str): a list of the root form of the article words
    """
        
    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", str(text).lower())
    
    # Tokenize text
    token = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in token if t not in stop]
    
    # Lemmatization
    lem = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lem

In [22]:
df["Preprocessed_Text"] = df['content summary'].apply(lambda x: preprocess(x))
df.head(10)

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name,Preprocessed_Text
0,0,Feb-22,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,geophysical event,"[mumbai, reuters, himalayan, avalanche, killed..."
1,1,Feb-22,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,geophysical event,"[zurich, reuters, one, person, killed, four, o..."
2,2,Feb-22,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,geophysical event,"[vienna, reuters, avalanche, area, austria, bo..."
3,3,Feb-22,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,geophysical event,"[new, delhi, reuters, himalayan, avalanche, tr..."
4,4,Feb-21,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,geophysical event,"[salt, lake, city, nytimes, four, back, countr..."
5,5,Apr-21,NEW DELHI (XINHUA),8 killed after glacial burst triggers avalanch...,https://www.straitstimes.com/asia/south-asia/8...,"According to officials, 384 others, mostly bel...","avalanche, officials, bro, glacial, uttarakhan...",natural calamities,geophysical event,"[according, official, others, mostly, belongin..."
6,6,Jan-21,VIENNA (REUTERS),Dogs' barking prompts owners' rescue from Swis...,https://www.straitstimes.com/world/europe/dogs...,VIENNA (REUTERS) - Two people caught in an ava...,"dogs, avalanche, buried, snowshoers, group, wi...",natural calamities,geophysical event,"[vienna, reuters, two, people, caught, avalanc..."
7,7,Jan-21,FRANCE (AFP),'Miracle' escape for man trapped in Alps avala...,https://www.straitstimes.com/world/europe/mira...,FRANCE (AFP) - A man out walking with his fami...,"minutes, avalanche, local, escape, snow, rescu...",natural calamities,geophysical event,"[france, afp, man, walking, family, french, al..."
8,8,Jan-20,"MUZAFFARABAD, PAKISTAN (REUTERS)",Girl buried under Pakistan avalanche for 18 ho...,https://www.straitstimes.com/asia/south-asia/g...,"MUZAFFARABAD, PAKISTAN (REUTERS) - A 12-year-o...","buried, avalanche, waited, pakistan, village, ...",natural calamities,geophysical event,"[muzaffarabad, pakistan, reuters, year, old, g..."
9,9,Dec-19,VIENNA (REUTERS),"Rescuers comb Austrian, Swiss avalanches in ca...",https://www.straitstimes.com/world/europe/resc...,VIENNA (REUTERS) - Rescuers hunted for possibl...,"avalanche, large, snow, warning, swiss, victim...",natural calamities,geophysical event,"[vienna, reuters, rescuer, hunted, possible, v..."


# Text Exploration

In [24]:
def find_common_words(df, category):
        
    """
    Function: find the most frequent words in the category and return the them
    Args:
      df(dataframe): the dataframe of articles
      category(str): the category name
    Return:
      the most frequant words in the category
    """
        
    # Create dataframes for the category
    cat_df = df[df["new_class_name"]==category]
    
    # Initialize words list for the category
    words = [word for tokens in cat_df["Preprocessed_Text"] for word in tokens]
    
    # Count words frequency
    words_counter = Counter(words)
 
    return words_counter.most_common(10)

In [25]:
print("Most common words in each category")
for c in category:
    print(c, " News")
    print(find_common_words(df, c))
    print()

Most common words in each category
geophysical event  News
[('said', 1426), ('people', 677), ('volcano', 580), ('eruption', 448), ('tsunami', 397), ('year', 391), ('earthquake', 386), ('mr', 379), ('island', 377), ('quake', 353)]

idiosyncratic  News
[('said', 1437), ('year', 579), ('singapore', 450), ('mr', 431), ('also', 382), ('food', 376), ('one', 368), ('safety', 356), ('people', 346), ('last', 328)]

pandemic  News
[('dengue', 955), ('said', 946), ('year', 780), ('case', 767), ('singapore', 583), ('health', 523), ('covid', 514), ('virus', 484), ('people', 477), ('pandemic', 445)]



In [26]:
df['Preprocessed_Text2'] = df['Preprocessed_Text'].apply(' '.join)
df.head()

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name,Preprocessed_Text,Preprocessed_Text2
0,0,Feb-22,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,geophysical event,"[mumbai, reuters, himalayan, avalanche, killed...",mumbai reuters himalayan avalanche killed seve...
1,1,Feb-22,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,geophysical event,"[zurich, reuters, one, person, killed, four, o...",zurich reuters one person killed four others i...
2,2,Feb-22,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,geophysical event,"[vienna, reuters, avalanche, area, austria, bo...",vienna reuters avalanche area austria borderin...
3,3,Feb-22,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,geophysical event,"[new, delhi, reuters, himalayan, avalanche, tr...",new delhi reuters himalayan avalanche trapped ...
4,4,Feb-21,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,geophysical event,"[salt, lake, city, nytimes, four, back, countr...",salt lake city nytimes four back country skier...


In [34]:
X = df['Preprocessed_Text2']
y = df['new_class_name']

# Feature Extraction

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [36]:
tf_vec = TfidfVectorizer()
train_features = tf_vec.fit(X_train)
train_features = tf_vec.transform(X_train)
test_features = tf_vec.transform(X_test)

# Modelling

In [37]:
def fit_eval_model(model, train_features, y_train, test_features, y_test):
    
    """
    Function: train and evaluate a machine learning classifier.
    Args:
      model: machine learning classifier
      train_features: train data extracted features
      y_train: train data lables
      test_features: train data extracted features
      y_test: train data lables
    Return:
      results(dictionary): a dictionary of the model training time and classification report
    """
    results ={}
    
    # Start time
    start = time.time()
    # Train the model
    model.fit(train_features, y_train)
    # End time
    end = time.time()
    # Calculate the training time
    results['train_time'] = end - start
    
    # Test the model
    train_predicted = model.predict(train_features)
    test_predicted = model.predict(test_features)
    
     # Classification report
    results['classification_report'] = classification_report(y_test, test_predicted)
        
    return results

In [38]:
sv = svm.SVC()
ab = AdaBoostClassifier(random_state = 1)
gb = GradientBoostingClassifier(random_state = 1)
xgb = xgboost.XGBClassifier(random_state = 1)
tree = DecisionTreeClassifier()
nb = MultinomialNB()


# Fit and evaluate models
results = {}
for cls in [sv, ab, gb, xgb, tree, nb]:
    cls_name = cls.__class__.__name__
    results[cls_name] = {}
    results[cls_name] = fit_eval_model(cls, train_features, y_train, test_features, y_test)

C:\Users\Darren\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:11:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [39]:
for res in results:
    print (res)
    print()
    for i in results[res]:
        print (i, ':')
        print(results[res][i])
        print()
    print ('-----')
    print()

SVC

train_time :
4.862982273101807

classification_report :
                   precision    recall  f1-score   support

geophysical event       0.95      0.77      0.85       217
    idiosyncratic       0.75      0.95      0.84       279
         pandemic       0.96      0.79      0.86       203

         accuracy                           0.85       699
        macro avg       0.89      0.84      0.85       699
     weighted avg       0.87      0.85      0.85       699


-----

AdaBoostClassifier

train_time :
0.9683742523193359

classification_report :
                   precision    recall  f1-score   support

geophysical event       0.92      0.72      0.81       217
    idiosyncratic       0.72      0.92      0.80       279
         pandemic       0.91      0.76      0.83       203

         accuracy                           0.81       699
        macro avg       0.85      0.80      0.81       699
     weighted avg       0.83      0.81      0.81       699


-----

GradientBoosti

# Using the model to predict new articles

### Creating a function

In [59]:
def classify_article(path):
    
    """
    Function: classify an article.
    Args:
      path: the path of the article 
    Return:
      category (str): the category of the article
    """
    # Read file
    file = open(path, 'r')
    artcl = file.read()

    # Text preprocessing
    artcl = preprocess(artcl)
    artcl = ' '.join(artcl)

    # Use TF_IDF
    test = tf_vec.transform([artcl])

    # Use MultinomialNB model to classify the article
    predict = gb.predict(test)
    category = predict[0]

    # Close file
    file.close()

    return category

In [60]:
print(classify_article('art1.txt'))

idiosyncratic


In [61]:
print(classify_article('art2.txt'))

pandemic


### Testing on new straits times article

In [62]:
from newspaper import Article

article = Article('https://www.straitstimes.com/singapore/environment/from-food-waste-to-medical-materials-0')


article.download()

article.parse()

article.nlp()

In [63]:
print(article.summary)

The ongoing Covid-19 pandemic has exposed two aspects of medical materials: the increased waste of medical supply (gloves) and the improved functionality of protective materials (masks).
Another seemingly unrelated source of environment pollution is from the food processing industry.
Food waste processing and treatment through incinerator and landfill in turn results in a higher carbon footprint.
To reduce global food waste and the global carbon footprint, food tech innovations have been developed to recover nutrients and convert the solid residues to usable materials.
Such innovations have unexpectedly connected the two sources of environment pollution: food processing side-streams and medical materials.


In [64]:
with open('art3.txt', 'w') as f:
    f.write(article.summary)